# Generating handwriting with Clockwork-RNN
This is the implementation of handwriting generation with Clockwork-RNN(CWRNN) in Mixture Density Network(MDN).

The main idea behind MDN is to use neural network outputs to parameterise a mixture distribution.

The idea behind CWRNN is to use more RNN cells but not to use all of them in every timestep. Cells in CWRNN have periods which tell us in what timestep the cell should be active.

This implemetation is heavily influenced by these projects:
*  https://github.com/aidangomez/models/tree/684883e4d4f310e59da109ba28cda8ba5f7785c9/clockwork_rnn
*  https://github.com/hardmaru/write-rnn-tensorflow

And these papers:
*  https://arxiv.org/pdf/1308.0850.pdf
*  https://arxiv.org/pdf/1402.3511.pdf

Loading data from xml files. Data can be found at http://www.fki.inf.unibe.ch/databases/iam-on-line-handwriting-database. I only used 300 files of total 12000 for time convenience. Path to data is saved in var $path$.

In [1]:
import os
import sys
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math
import random
import time

path = "C:/Users/Jirka/Downloads/lineStrokes"
filenames = []
for dir1 in os.listdir(path):
    for dir2 in os.listdir(path+"/"+dir1):
        for filename in os.listdir(path+"/"+dir1+"/"+dir2):
            filenames.append(path+"/"+dir1+"/"+dir2+"/"+filename)
            
trainSequences = []
            
for filename in filenames:  
    root = ET.parse(filename).getroot();
    XY = root.find('WhiteboardDescription')

    maxX = (float)(XY.find('DiagonallyOppositeCoords').get('x'))
    maxY = (float)(XY.find('DiagonallyOppositeCoords').get('y'))
    minX = (float)(XY.find('VerticallyOppositeCoords').get('x'))
    minY = (float)(XY.find('HorizontallyOppositeCoords').get('y'))

    line = []
    for stroke in root.iter('Stroke'):
        sTime = (float)(stroke.get('start_time'))
        eTime = (float)(stroke.get('end_time'))
        dTime = ((eTime-sTime) if (sTime-eTime)>0 else 1)
        for point in stroke.iter('Point'):
            line.append([int(point.get('x')), int(point.get('y')), 0]);
        line[-1][2] = 1
    
    for it in range(len(line)-1, 0, -1):
        line[it] = [line[it][0]-line[it-1][0],
                    line[it][1]-line[it-1][1],
                    line[it][2]]
    line[0] = [0, 0, 0]
    trainSequences.append(line)

npTrainSequences = []
for line in trainSequences:
    npTrainSequences.append(np.array(line, dtype=np.int16))
    
    
valid_data = []
train_data = []
counter = 0
cur_data_counter = 0

for data in npTrainSequences:
    if len(data) > (300+2):
        # removes large gaps from the data
        data = np.minimum(data, 500)
        data = np.maximum(data, -500)
        data = np.array(data,dtype=np.float32)
        data[:,0:2] /= 10
        cur_data_counter = cur_data_counter + 1
        if cur_data_counter % 20 == 0:
            valid_data.append(data)
        else:
            train_data.append(data)
            counter += int(len(data)/((300+2))) # number of equiv batches this datapoint is worth
            
print("train data: {}, valid data: {}".format(len(train_data), len(valid_data)))
    
size = 0
for a in npTrainSequences:
    size += sys.getsizeof(a)

#Data loader prepared for batch fetching to neural network.
class DataLoader():
    def __init__(self, train_data, valid_data, batch_size=50):
        self.seq_length = 300
        self.train_data = train_data
        self.valid_data = valid_data
        self.batch_size = batch_size
        self.reset_batch_pointer()
        self.num_batches = int(counter / self.batch_size)
    
    def next_batch(self):
        # returns a randomised, seq_length sized portion of the training data
        x_batch = []
        y_batch = []
        for i in range(self.batch_size):
            data = self.train_data[self.pointer]
            n_batch = int(len(data)/((self.seq_length+2))) # number of equiv batches this datapoint is worth
            idx = random.randint(0, len(data)-self.seq_length-2)
            x_batch.append(np.copy(data[idx:idx+self.seq_length]))
            y_batch.append(np.copy(data[idx+1:idx+self.seq_length+1]))
            if random.random() < (1.0/float(n_batch)): # adjust sampling probability.
                #if this is a long datapoint, sample this data more with higher probability
                self.pointer += 1
                if (self.pointer >= len(self.train_data)):
                    self.pointer = 0
        return np.array(x_batch), np.array(y_batch)
    
    def validation_data(self):
        # returns validation data
        x_batch = []
        y_batch = []
        for i in range(self.batch_size):
            data = self.valid_data[i%len(self.valid_data)]
            idx = 0
            x_batch.append(np.copy(data[idx:idx+self.seq_length]))
            y_batch.append(np.copy(data[idx+1:idx+self.seq_length+1]))
        return x_batch, y_batch

    def reset_batch_pointer(self):
        self.pointer = 0

train data: 11035, valid data: 580


Setting up network model

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from clockwork.cwrnn import CWRNNCell

batch_size = 50
seq_len = 300
num_components = 3

NCELLS = 9
NHIDDEN = 32*NCELLS

inputs = tf.placeholder(dtype=tf.float32, shape=[batch_size, seq_len, num_components])
targets = tf.placeholder(dtype=tf.float32, shape=[batch_size, seq_len, num_components])

#CWRNN Part
cells = []
for i in range(NCELLS):
    cells.append(tf.contrib.rnn.BasicRNNCell(num_units=NHIDDEN//NCELLS))

#periods = []
#for i in range(NCELLS):
#    periods.append(2**i)
    
periods = [1, 2, 4, 8, 16, 32, 64, 96, 128]

cell = CWRNNCell(cells, periods, state_is_tuple=False)

#LSTM Part
#cell = tf.contrib.rnn.BasicLSTMCell(NHIDDEN, state_is_tuple=False)

#cell = tf.contrib.rnn.MultiRNNCell(
#            [tf.contrib.rnn.BasicLSTMCell(NHIDDEN, state_is_tuple=False) for _ in range(2)],
#            state_is_tuple=False
#)

#cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob = 0.8)

zero_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
prev_state = tf.identity(zero_state, name='prev_state')

NOUT = 1 + 20 * 6 # end_of_stroke + prob + 2*(mu + sig) + corr

with tf.variable_scope('rnnlm'):
    output_w = tf.get_variable("output_w", [NHIDDEN, NOUT])
    output_b = tf.get_variable("output_b", [NOUT])
    
inputs2 = tf.unstack(inputs, axis=1)

#Forward pass
outputs, state_out = tf.contrib.legacy_seq2seq.rnn_decoder(inputs2, prev_state, cell, loop_function=None, scope='rnnlm')
output = tf.reshape(tf.concat(axis=1, values=outputs), [-1, NHIDDEN])
output = tf.nn.xw_plus_b(output, output_w, output_b)
state_out = tf.identity(state_out, name='state_out')

#Some trying

#outputs, _ = tf.contrib.legacy_seq2seq.rnn_decoder(cell=cell, decoder_inputs=x, initial_state=zero_state)
#outputs, out_state = tf.nn.dynamic_rnn(cell, inputs, initial_state=prev_state, time_major=False)

#NOUT = 1 + 20 * 6 # end_of_stroke + prob + 2*(mu + sig) + corr

#W_out = tf.Variable(tf.random_normal([NHIDDEN,NOUT], stddev=1.0, dtype=tf.float32))
#b_out = tf.Variable(tf.random_normal([NOUT], stddev=1.0, dtype=tf.float32))

#output = tf.reshape(outputs, [-1, NHIDDEN])
#output = tf.reshape(tf.concat(axis=1, values=outputs), [-1, NHIDDEN])
#print(output.shape)
#output = tf.nn.xw_plus_b(output, W_out, b_out)

#End of some trying


flat_targets = tf.reshape(targets, [-1, num_components])
[x1_data, x2_data, eos_data] = tf.split(axis=1, num_or_size_splits=3, value=flat_targets)

#MDN Part. Mainly copied from https://github.com/hardmaru/write-rnn-tensorflow.
def tf_2d_normal(x1, x2, mu1, mu2, s1, s2, rho):
    # eq # 24 and 25 of http://arxiv.org/abs/1308.0850
    norm1 = tf.subtract(x1, mu1)
    norm2 = tf.subtract(x2, mu2)
    s1s2 = tf.multiply(s1, s2)
    z = tf.square(tf.div(norm1, s1))+tf.square(tf.div(norm2, s2))-2*tf.div(tf.multiply(rho, tf.multiply(norm1, norm2)), s1s2)
    negRho = 1-tf.square(rho)
    result = tf.exp(tf.div(-z,2*negRho))
    denom = 2*np.pi*tf.multiply(s1s2, tf.sqrt(negRho))
    result = tf.div(result, denom)
    return result

def get_lossfunc(z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_eos, x1_data, x2_data, eos_data):
    result0 = tf_2d_normal(x1_data, x2_data, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr)
    # implementing eq # 26 of http://arxiv.org/abs/1308.0850
    epsilon = 1e-20
    result1 = tf.multiply(result0, z_pi)
    result1 = tf.reduce_sum(result1, 1, keep_dims=True)
    result1 = -tf.log(tf.maximum(result1, 1e-20)) # at the beginning, some errors are exactly zero.

    result2 = tf.multiply(z_eos, eos_data) + tf.multiply(1-z_eos, 1-eos_data)
    result2 = -tf.log(result2)

    result = result1 + result2
    return tf.reduce_sum(result)

# below is where we need to do MDN splitting of distribution params
def get_mixture_coef(output):
    # returns the tf slices containing mdn dist params
    # ie, eq 18 -> 23 of http://arxiv.org/abs/1308.0850
    z = output
    z_eos = z[:, 0:1]
    z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr = tf.split(axis=1, num_or_size_splits=6, value=z[:, 1:])

    # process output z's into MDN paramters

    # end of stroke signal
    z_eos = tf.sigmoid(z_eos) # should be negated, but doesn't matter.

    # softmax all the pi's:
    max_pi = tf.reduce_max(z_pi, 1, keep_dims=True)
    z_pi = tf.subtract(z_pi, max_pi)
    z_pi = tf.exp(z_pi)
    normalize_pi = tf.reciprocal(tf.reduce_sum(z_pi, 1, keep_dims=True))
    z_pi = tf.multiply(normalize_pi, z_pi)

    # exponentiate the sigmas and also make corr between -1 and 1.
    z_sigma1 = tf.exp(z_sigma1)
    z_sigma2 = tf.exp(z_sigma2)
    z_corr = tf.tanh(z_corr)

    return [z_pi, z_mu1, z_mu2, z_sigma1, z_sigma2, z_corr, z_eos]

#Get distribution parameters from network output
[o_pi, o_mu1, o_mu2, o_sigma1, o_sigma2, o_corr, o_eos] = get_mixture_coef(output)

#Parameters for outputs
data_out_pi = tf.identity(o_pi, "data_out_pi");
data_out_mu1 = tf.identity(o_mu1, "data_out_mu1");
data_out_mu2 = tf.identity(o_mu2, "data_out_mu2");
data_out_sigma1 = tf.identity(o_sigma1, "data_out_sigma1");
data_out_sigma2 = tf.identity(o_sigma2, "data_out_sigma2");
data_out_corr = tf.identity(o_corr, "data_out_corr");
data_out_eos = tf.identity(o_eos, "data_out_eos");

#Loss Function
lossfunc = get_lossfunc(o_pi, o_mu1, o_mu2, o_sigma1, o_sigma2, o_corr, o_eos, x1_data, x2_data, eos_data)
cost = lossfunc / (batch_size * seq_len)

#Train network with AdamOptimizer and desired learninig rate with gradient clippingg
lr = tf.Variable(0.0, trainable=False)
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), 10.0)
optimizer = tf.train.AdamOptimizer(lr)
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [3]:
model_path = "C:\\Users\\Jirka\\Desktop\\CWRNN_TRY\\"

sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.global_variables())

data_loader = DataLoader(train_data, valid_data, batch_size)
NEPOCH = 100
for e in range(NEPOCH):
    sess.run(tf.assign(lr, 0.005 * (0.95 ** e)))
    data_loader.reset_batch_pointer()
    v_x, v_y = data_loader.validation_data()
    valid_feed = {inputs: v_x, targets: v_y, prev_state: prev_state.eval()}
    
    state = prev_state.eval()
    for b in range(data_loader.num_batches):
        i = e * data_loader.num_batches + b
        start = time.time()
        x, y = data_loader.next_batch()
        feed = {inputs: x, targets: y, prev_state: state}
        train_loss, state, _ = sess.run([cost, state_out, train_op], feed)
        valid_loss, = sess.run([cost], valid_feed)

        end = time.time()
        print(
            "{}/{} (epoch {}), train_loss = {:.3f}, valid_loss = {:.3f}, time/batch = {:.3f}"  \
            .format(
                i,
                NEPOCH * data_loader.num_batches,
                e, 
                train_loss, valid_loss, end - start))
        if (e * data_loader.num_batches + b) % 1000 == 0 and ((e * data_loader.num_batches + b) > 0):
            checkpoint_path = os.path.join(model_path, 'model.ckpt')
            saver.save(sess, checkpoint_path, global_step = e * data_loader.num_batches + b)
            print("model saved to {}".format(checkpoint_path))

sess.close()

0/36400 (epoch 0), train_loss = 6.013, valid_loss = 5.609, time/batch = 10.944
1/36400 (epoch 0), train_loss = 4.916, valid_loss = 5.218, time/batch = 0.465
2/36400 (epoch 0), train_loss = 4.729, valid_loss = 4.917, time/batch = 0.456
3/36400 (epoch 0), train_loss = 4.529, valid_loss = 4.668, time/batch = 0.487
4/36400 (epoch 0), train_loss = 4.310, valid_loss = 4.472, time/batch = 0.455
5/36400 (epoch 0), train_loss = 4.265, valid_loss = 4.309, time/batch = 0.459
6/36400 (epoch 0), train_loss = 4.194, valid_loss = 4.175, time/batch = 0.458
7/36400 (epoch 0), train_loss = 3.778, valid_loss = 4.052, time/batch = 0.562
8/36400 (epoch 0), train_loss = 3.719, valid_loss = 3.947, time/batch = 0.548
9/36400 (epoch 0), train_loss = 3.701, valid_loss = 3.850, time/batch = 0.541
10/36400 (epoch 0), train_loss = 3.895, valid_loss = 3.785, time/batch = 0.513
11/36400 (epoch 0), train_loss = 3.403, valid_loss = 3.702, time/batch = 0.540
12/36400 (epoch 0), train_loss = 3.711, valid_loss = 3.620, t

104/36400 (epoch 0), train_loss = 2.153, valid_loss = 2.404, time/batch = 0.441
105/36400 (epoch 0), train_loss = 1.898, valid_loss = 2.389, time/batch = 0.465
106/36400 (epoch 0), train_loss = 1.997, valid_loss = 2.381, time/batch = 0.451
107/36400 (epoch 0), train_loss = 2.264, valid_loss = 2.369, time/batch = 0.459
108/36400 (epoch 0), train_loss = 1.591, valid_loss = 2.358, time/batch = 0.469
109/36400 (epoch 0), train_loss = 2.145, valid_loss = 2.354, time/batch = 0.462
110/36400 (epoch 0), train_loss = 1.561, valid_loss = 2.342, time/batch = 0.445
111/36400 (epoch 0), train_loss = 1.992, valid_loss = 2.334, time/batch = 0.439
112/36400 (epoch 0), train_loss = 2.136, valid_loss = 2.336, time/batch = 0.443
113/36400 (epoch 0), train_loss = 2.046, valid_loss = 2.343, time/batch = 0.441
114/36400 (epoch 0), train_loss = 2.196, valid_loss = 2.350, time/batch = 0.447
115/36400 (epoch 0), train_loss = 1.596, valid_loss = 2.364, time/batch = 0.444
116/36400 (epoch 0), train_loss = 1.943,

207/36400 (epoch 0), train_loss = 1.902, valid_loss = 2.245, time/batch = 0.533
208/36400 (epoch 0), train_loss = 1.863, valid_loss = 2.239, time/batch = 0.519
209/36400 (epoch 0), train_loss = 1.689, valid_loss = 2.245, time/batch = 0.502
210/36400 (epoch 0), train_loss = 1.766, valid_loss = 2.250, time/batch = 0.538
211/36400 (epoch 0), train_loss = 1.561, valid_loss = 2.274, time/batch = 0.470
212/36400 (epoch 0), train_loss = 1.335, valid_loss = 2.301, time/batch = 0.469
213/36400 (epoch 0), train_loss = 1.645, valid_loss = 2.303, time/batch = 0.456
214/36400 (epoch 0), train_loss = 1.930, valid_loss = 2.293, time/batch = 0.481
215/36400 (epoch 0), train_loss = 1.673, valid_loss = 2.288, time/batch = 0.543
216/36400 (epoch 0), train_loss = 2.205, valid_loss = 2.302, time/batch = 0.463
217/36400 (epoch 0), train_loss = 1.827, valid_loss = 2.296, time/batch = 0.455
218/36400 (epoch 0), train_loss = 2.026, valid_loss = 2.284, time/batch = 0.526
219/36400 (epoch 0), train_loss = 1.698,

310/36400 (epoch 0), train_loss = 1.873, valid_loss = 2.225, time/batch = 0.494
311/36400 (epoch 0), train_loss = 1.952, valid_loss = 2.222, time/batch = 0.474
312/36400 (epoch 0), train_loss = 1.700, valid_loss = 2.222, time/batch = 0.461
313/36400 (epoch 0), train_loss = 1.474, valid_loss = 2.214, time/batch = 0.476
314/36400 (epoch 0), train_loss = 1.677, valid_loss = 2.221, time/batch = 0.464
315/36400 (epoch 0), train_loss = 1.747, valid_loss = 2.225, time/batch = 0.509
316/36400 (epoch 0), train_loss = 1.763, valid_loss = 2.219, time/batch = 0.525
317/36400 (epoch 0), train_loss = 2.133, valid_loss = 2.221, time/batch = 0.531
318/36400 (epoch 0), train_loss = 1.488, valid_loss = 2.219, time/batch = 0.490
319/36400 (epoch 0), train_loss = 1.773, valid_loss = 2.220, time/batch = 0.558
320/36400 (epoch 0), train_loss = 1.727, valid_loss = 2.222, time/batch = 0.518
321/36400 (epoch 0), train_loss = 1.808, valid_loss = 2.226, time/batch = 0.467
322/36400 (epoch 0), train_loss = 1.522,

413/36400 (epoch 1), train_loss = 1.942, valid_loss = 2.066, time/batch = 0.494
414/36400 (epoch 1), train_loss = 1.498, valid_loss = 2.069, time/batch = 0.452
415/36400 (epoch 1), train_loss = 1.917, valid_loss = 2.067, time/batch = 0.469
416/36400 (epoch 1), train_loss = 1.879, valid_loss = 2.077, time/batch = 0.458
417/36400 (epoch 1), train_loss = 1.594, valid_loss = 2.096, time/batch = 0.534
418/36400 (epoch 1), train_loss = 1.940, valid_loss = 2.094, time/batch = 0.511
419/36400 (epoch 1), train_loss = 1.511, valid_loss = 2.078, time/batch = 0.493
420/36400 (epoch 1), train_loss = 1.637, valid_loss = 2.077, time/batch = 0.563
421/36400 (epoch 1), train_loss = 1.870, valid_loss = 2.067, time/batch = 0.512
422/36400 (epoch 1), train_loss = 1.654, valid_loss = 2.057, time/batch = 0.573
423/36400 (epoch 1), train_loss = 1.762, valid_loss = 2.060, time/batch = 0.520
424/36400 (epoch 1), train_loss = 1.684, valid_loss = 2.057, time/batch = 0.525
425/36400 (epoch 1), train_loss = 1.662,

...

20364/36400 (epoch 55), train_loss = 1.279, valid_loss = 1.798, time/batch = 0.457
20365/36400 (epoch 55), train_loss = 1.320, valid_loss = 1.798, time/batch = 0.458
20366/36400 (epoch 55), train_loss = 1.426, valid_loss = 1.798, time/batch = 0.449
20367/36400 (epoch 55), train_loss = 1.510, valid_loss = 1.798, time/batch = 0.449
20368/36400 (epoch 55), train_loss = 1.869, valid_loss = 1.798, time/batch = 0.448
20369/36400 (epoch 55), train_loss = 1.202, valid_loss = 1.798, time/batch = 0.452
20370/36400 (epoch 55), train_loss = 1.528, valid_loss = 1.797, time/batch = 0.447
20371/36400 (epoch 55), train_loss = 1.457, valid_loss = 1.796, time/batch = 0.445
20372/36400 (epoch 55), train_loss = 1.452, valid_loss = 1.794, time/batch = 0.444
20373/36400 (epoch 55), train_loss = 1.347, valid_loss = 1.794, time/batch = 0.442
20374/36400 (epoch 55), train_loss = 0.961, valid_loss = 1.794, time/batch = 0.448
20375/36400 (epoch 55), train_loss = 1.589, valid_loss = 1.794, time/batch = 0.442
2037

20463/36400 (epoch 56), train_loss = 1.390, valid_loss = 1.786, time/batch = 0.477
20464/36400 (epoch 56), train_loss = 1.527, valid_loss = 1.786, time/batch = 0.530
20465/36400 (epoch 56), train_loss = 1.829, valid_loss = 1.786, time/batch = 0.570
20466/36400 (epoch 56), train_loss = 0.938, valid_loss = 1.786, time/batch = 0.543
20467/36400 (epoch 56), train_loss = 1.329, valid_loss = 1.786, time/batch = 0.495
20468/36400 (epoch 56), train_loss = 1.240, valid_loss = 1.787, time/batch = 0.507
20469/36400 (epoch 56), train_loss = 1.556, valid_loss = 1.788, time/batch = 0.534
20470/36400 (epoch 56), train_loss = 1.534, valid_loss = 1.790, time/batch = 0.491
20471/36400 (epoch 56), train_loss = 1.354, valid_loss = 1.793, time/batch = 0.483
20472/36400 (epoch 56), train_loss = 1.295, valid_loss = 1.796, time/batch = 0.475
20473/36400 (epoch 56), train_loss = 1.086, valid_loss = 1.797, time/batch = 0.453
20474/36400 (epoch 56), train_loss = 1.685, valid_loss = 1.796, time/batch = 0.469
2047

20562/36400 (epoch 56), train_loss = 1.487, valid_loss = 1.794, time/batch = 0.590
20563/36400 (epoch 56), train_loss = 1.560, valid_loss = 1.792, time/batch = 0.548
20564/36400 (epoch 56), train_loss = 1.448, valid_loss = 1.790, time/batch = 0.550
20565/36400 (epoch 56), train_loss = 1.481, valid_loss = 1.789, time/batch = 0.536
20566/36400 (epoch 56), train_loss = 1.596, valid_loss = 1.787, time/batch = 0.471
20567/36400 (epoch 56), train_loss = 1.528, valid_loss = 1.787, time/batch = 0.481
20568/36400 (epoch 56), train_loss = 1.727, valid_loss = 1.788, time/batch = 0.509
20569/36400 (epoch 56), train_loss = 1.726, valid_loss = 1.789, time/batch = 0.512
20570/36400 (epoch 56), train_loss = 1.842, valid_loss = 1.789, time/batch = 0.500
20571/36400 (epoch 56), train_loss = 1.409, valid_loss = 1.789, time/batch = 0.536
20572/36400 (epoch 56), train_loss = 1.523, valid_loss = 1.789, time/batch = 0.511
20573/36400 (epoch 56), train_loss = 1.428, valid_loss = 1.789, time/batch = 0.519
2057

20661/36400 (epoch 56), train_loss = 1.029, valid_loss = 1.801, time/batch = 0.512
20662/36400 (epoch 56), train_loss = 1.404, valid_loss = 1.801, time/batch = 0.485
20663/36400 (epoch 56), train_loss = 1.163, valid_loss = 1.802, time/batch = 0.494
20664/36400 (epoch 56), train_loss = 1.410, valid_loss = 1.803, time/batch = 0.472
20665/36400 (epoch 56), train_loss = 1.157, valid_loss = 1.805, time/batch = 0.471


KeyboardInterrupt: 